In [26]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd
import numpy as np
import datetime
import random
import concurrent.futures
from geopy.geocoders import Nominatim

# --------------------------------------------
# Read recipe inputs
activity_log_with_label = dataiku.Dataset("activity_log_with_label")
activity_df = activity_log_with_label.get_dataframe()

location_map = dataiku.Dataset("location_map")
location_map_df = location_map.get_dataframe()

# --------------------------------------------
# Generate random timestamp between given dates
def random_timestamp():
    start_datetime = datetime.datetime(2024, 4, 1, 0, 0, 0)      # April 1, 2024 00:00:00
    end_datetime = datetime.datetime(2025, 4, 30, 23, 59, 59)   # April 30, 2025 23:59:59

    time_between = end_datetime - start_datetime
    seconds_between = time_between.total_seconds()
    random_seconds = random.randrange(int(seconds_between))
    random_dt = start_datetime + datetime.timedelta(seconds=random_seconds)
    return random_dt

# Add access_timestamp column with random timestamps
activity_df['access_timestamp'] = [random_timestamp() for _ in range(len(activity_df))]

In [28]:
location_map_df.head()

city  risky_ind    alt_city
0  New York        0.0  Newark, NJ
1    London        1.0     Reading
2     Tokyo        0.0    Yokohama
3     Paris        0.0  Versailles
4    Berlin        0.0     Potsdam

In [29]:
location_map_df.columns

Index(['city', 'risky_ind', 'alt_city'], dtype='object')

In [35]:
location_map_df

# Create the safe_cities list (risky_ind == 0)
safe_cities = location_map_df[location_map_df['risky_ind'] == 0]['city'].unique().tolist()

# Create the risky_cities list (risky_ind == 1)
risky_cities = location_map_df[location_map_df['risky_ind'] == 1]['city'].unique().tolist()


In [38]:
# Assign location based on synthetic flag
np.random.seed(42)
def assign_location(row):
    if row['is_synthetic'] == 1:
        return np.random.choice(risky_cities)
    else:
        return np.random.choice(safe_cities)
activity_df['location'] = activity_df.apply(assign_location, axis=1)

In [0]:
# Output dataset write
location_translate_df = activity_df

location_translate = dataiku.Dataset("location_translate")
location_translate.write_with_schema(location_translate_df)